In [72]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import logging
from bs4 import BeautifulSoup
from datetime import datetime  
from datetime import timedelta 

weekday_mapper = {
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}


#logging.basicConfig(filename='imse.log', level=logging.INFO)
logging.basicConfig(level=logging.INFO)

today = datetime.today()
then = today + timedelta(days=14)

logging.info(f"It is {weekday_mapper[today.weekday()]}, so Imse is waking up, acting on (alias) {alias}s behalf!\n")
year = then.year
month = then.month
day = then.day
book_url = f"""
https://www.matchi.se/facilities/nynashamnpadelcenter?date={year}-{month}-{day}&sport=
"""

login_url = "https://www.matchi.se/login/auth?returnUrl=%2Ffacilities%2Fnynashamnpadelcenter"

INFO:root:It is Sunday, so Imse is waking up, acting on (alias) V Ws behalf!



In [80]:
logging.info(f"Logging in at {login_url}...")
logging.info(f"Moving to the correct date at {book_url}...")

try:
    logging.info(f"IMSE ACTIVATED AT {str(datetime.now())}")
    driver = webdriver.Chrome()
    driver.get(login_url)
    assert driver.title == 'Login - MATCHi'
    logging.info("Logging in...")

    username = driver.find_element_by_id("username")
    username.clear()
    username.send_keys(uname)

    password = driver.find_element_by_name("j_password")
    password.clear()
    password.send_keys(pwd)

    driver.find_element_by_xpath('//*[@id="loginForm"]/button').click()
    time.sleep(1)

    driver.get(book_url)
    driver.fullscreen_window()

    time.sleep(1)
    logging.info("Looking for free slots...")

    soup = BeautifulSoup(driver.page_source, 'lxml')
    time.sleep(1)
    schedule = soup.find('div', attrs = {'class': 'schedule'})
    free = schedule.find_all('td', attrs = {'class': 'slot free'})
    
    slots = pd.DataFrame()
    for i, f in enumerate(free):
        slot_id = f['slotid']
        slot_title = f['data-original-title']
        slot_availability, slot_lane, slot_time = slot_title.split(sep='<br>')
        status, name, slottime = slot_title.split(sep ='<br>')
        _, start, _, end = slottime.split(sep = ' ')

        name = name.split(sep = '.')[1]

        slots.at[i, "SLOT_ID"] = slot_id
        slots.at[i, "NAME"] = name
        slots.at[i, "START"] = start
        slots.at[i, "END"] = end
    logging.info(f"Found {slots}")


    time_priority = ['15:00', '16:00', '14:00']

    for i, p in enumerate(time_priority):
        matched_slots = slots[slots['START'] == p].shape[0]
        logging.info(f"Looking at priority {i + 1} timeslot, {p}, and found {matched_slots} possibilities")

        if matched_slots > 0:
            for slot_index in range(matched_slots):
                selected_booking_data = slots[slots['START'] == p].iloc[slot_index]
                logging.info(f"Selecting {selected_booking_data}")
                SELECTED_ID = selected_booking_data.SLOT_ID
                try:
                    selected_slot = driver.find_element_by_css_selector(f"td[slotid='{SELECTED_ID}']")
                    logging.info(f"Attempting to book {SELECTED_ID} at {str(datetime.now())}")
                    selected_slot.click()
                    time.sleep(1)

                    driver.find_element_by_id('btnSubmit').click()
                    break;
                except Exception as e:
                    logging.warning(f"Couln't book that, continuing... {e}")


    time.sleep(1)
    driver.close()
except Exception as e:
    logging.warning(f"Imse was Vimse and fell down the drain: / {e}")
    driver.close()

INFO:root:Logging in at https://www.matchi.se/login/auth?returnUrl=%2Ffacilities%2Fnynashamnpadelcenter...
INFO:root:Moving to the correct date at 
https://www.matchi.se/facilities/nynashamnpadelcenter?date=2021-2-28&sport=
...
INFO:root:IMSE ACTIVATED AT 2021-02-14 20:28:36.761662
INFO:root:Logging in...
INFO:root:Looking for free slots...
INFO:root:Found                             SLOT_ID                             NAME  START  \
0  e4e067c675e9c5a00175f06e5aac0148               Monster HydroSport  06:00   
1  e4e067c675e9c5a00175f06e5aae014f               Monster HydroSport  07:00   
2  e4e11241766a92b3017672d5f24873b4               Monster HydroSport  22:30   
3  e4e067c675e9c5a00175f06e5ad001cd                        Lantchips  06:00   
4  e4e067c675e9c5a00175f06e5af2024c                        Lantchips  22:00   
5  e4e11241766a92b3017672968a866781                        Lantchips  23:00   
6  e4e067c675e9c5a00175f06e5af50258   Skärgårdens ICA MAXI Nynäshamn  06:00   
7  e4e067

In [69]:
email = f"It is {weekday_mapper[today.weekday()]}, so Imse is waking up, acting on (alias) {alias}s behalf!\n"
email += f"The time is {today}, and imse will look for free timeslots at {str(then)[:10]}\n"
email += f"Imse will prioritize timeslots starting at {time_priority} in that order"
print(email)

It is Sunday, so Imse is waking up, acting on (alias) V Ws behalf!
The time is 2021-02-14 20:13:26.736049, and imse will look for free timeslots at 2021-02-28
Imse will prioritize timeslots starting at ['15:00', '16:00', '14:00'] in that order
